In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import six

# Load in the stored data

In [36]:
DATA_FILE = 'four_players_data.csv'

In [37]:
df_all = pd.read_csv(DATA_FILE)
print('Original sample size', len(df_all))

Original sample size 6714


# Helper methods

In [38]:
def all_players(key):
    return ['pos_{}{}'.format(i, key) for i in range(1,5)]

In [39]:
def field_search(key, search_df=df_all):
    return {_ for _ in search_df if key in _}

In [40]:
def get_faction_df(df, faction):
    df_list = []
    for i in range(1,5) :
        col = 'pos_{}{}'.format(i, '_faction')
        pos_df = df.loc[df[col] == faction]
        col_dict = {}
        for pos_field in field_search('pos_{}'.format(i),df):
            target_field = 'target' + pos_field[5:]
            col_dict[target_field] = pos_df[pos_field]
        target_df = pd.concat(col_dict, axis=1)
        target_df = pd.concat([pos_df, target_df], axis=1) 
        df_list.append(target_df)
    df_faction = pd.concat(df_list)
    # display(df_faction['target_score'])
    df_faction['target_score'] = df_faction['target_score'] + df_faction['target_bid']
    # display(df_faction['target_score'])
    # display(df_faction['target_score_bid'].value_counts())
    return df_faction

In [41]:
def getTableHTML(df):
    
    """
    From https://stackoverflow.com/a/49687866/2007153
    
    Get a Jupyter like html of pandas dataframe
    
    """

    
    styles = [
        #table properties
        dict(selector=" ", 
             props=[("margin","0"),
                    ("font-family",'"Helvetica", "Arial", sans-serif'),
                    ("border-collapse", "collapse"),
                    ("border", "none"),
                    ("border","1px solid #ddd")
                       ]),

        #header color - optional
        dict(selector="thead", 
             props=[("background-color","#fff")
                   ]),

        #background shading
        dict(selector="tbody tr:nth-child(even)",
             props=[("background-color", "#cde")]),
        dict(selector="tbody tr:nth-child(odd)",
             props=[("background-color", "#eee")]),

        #cell spacing
        dict(selector="td", 
             props=[("padding", "1.0em")]),

        #header cell properties
        dict(selector="th", 
             props=[("font-size", "100%"),
                    ("text-align", "center")]),


    ]
    return (df.style.set_table_styles(styles).format({col: "{:.2f}" for col in assembled_df.columns[:-3]}))

# Search Example

In [43]:
look_cols = list(field_search('fed') - (field_search('pos_') - field_search('pos_1')))
look_cols

['pos_1_fed_12vp',
 'pos_1_fed_2o',
 'pos_1_adv_tech_taken_3vp_per_fed_pass',
 'pos_1_adv_tech_taken_5vp_per_fed',
 'pos_1_feds_taken',
 'pos_1_fed_gleens',
 'pos_1_fed_2pw',
 'pos_1_score_fed',
 'pos_1_fed_qic',
 'pos_1_fed_6c',
 'pos_1_fed_2k']

# Basic Data Cleaning

In [44]:
# Fields with one column for each player
PLAYER_DROP_FIELDS = ['_research_level_dip']

# Fields with only one column for the table
SINGLE_DROP_FIELDS = ['Unnamed: 0']

drop_col_list = []
for field in PLAYER_DROP_FIELDS:
    drop_col_list += all_players(field)
drop_col_list += SINGLE_DROP_FIELDS
df_cleaned = df_all.drop(columns=drop_col_list,errors='ignore')

In [45]:
print('Sample size after cleaning', len(df_cleaned))

Sample size after cleaning 6714


# Filtering

In [46]:
VARIANTS = ['standard'] # Possible values: ['standard','more-balanced','beta']

MAP_LAYOUTS = ['standard'] # Possible to include 'xshape', but doesn't make sense for 4p

# 4 player games only
df_filtered = df_cleaned.loc[df_cleaned.num_players == 4]

# Filter variants
df_filtered = df_filtered.loc[df_filtered.balance_variant.isin(VARIANTS)]

# Filter map layout
df_filtered = df_filtered.loc[df_filtered.map_layout.isin(MAP_LAYOUTS)]

# Remove games where any player is dropped
for col in all_players('_dropped'):
    df_filtered = df_filtered.loc[(df_filtered[col] == False)]


In [47]:
print('Sample size after filtering', len(df_filtered))

Sample size after filtering 4323


# R1 Actions

In [48]:
faction_list = df_cleaned.pos_1_faction.unique()
assert(len(faction_list) == 14)

In [55]:
field_search('power3').intersection(field_search('r_1'))

{'pos_1_action_r_1_power3',
 'pos_2_action_r_1_power3',
 'pos_3_action_r_1_power3',
 'pos_4_action_r_1_power3'}

In [ ]:
look_cols = list(field_search('power') - (field_search('pos_') - field_search('pos_1')))

In [59]:
pd.options.display.float_format = '{:.2}'.format
for i in [3,2,1]:
    full_dict = {}
    sort_key = '% took 2o action in R{}'.format(i)
    for faction in faction_list:
        faction_dict = {}
        faction_df = get_faction_df(df_filtered, faction)
        r_2o_df = faction_df[faction_df['target_action_r_{}_power3'.format(i)] == 1]
        faction_dict[sort_key] = len(r_2o_df)/len(faction_df) * 100
        faction_dict['additional +pts if made if 2o in R{}'.format(i)] = r_2o_df.target_score.mean() - faction_df.target_score.mean()
        # faction_dict['+pts if 5vp/fed scoring R{}'.format(i)] = fed_scoring.target_score.mean() - faction_df.target_score.mean()
        # faction_dict['overall avg score'] = faction_df.target_score.mean()
        # faction_dict['avg score if 5vp/fed scoring R{}'.format(i)] = fed_scoring.target_score.mean()
        # faction_dict['avg score if made fed in 5vp/fed scoring R{}'.format(i)] = made_fed.target_score.mean()
        # faction_dict['% players pick if 5vp/fed scoring R{}'.format(i)] = ((fed_scoring.shape[0]/faction_df.shape[0])*100)
        # faction_dict['total games'] = faction_df.shape[0]
        # faction_dict['games with 5vp/fed scoring R{}'.format(i)] = fed_scoring.shape[0]
        # faction_dict['games fed made in 5vp/fed scoring R{}'.format(i)] = made_fed.shape[0]
        full_dict[faction] = faction_dict
    assembled_df = pd.DataFrame.from_dict(full_dict,orient='index').sort_values(by=sort_key, ascending=False)
    html_df = getTableHTML(assembled_df)
    
    with open('r{}_2o.html'.format(i), 'w') as fo:
        fo.write(html_df.to_html())
html_df

,% took 2o action in R1,additional +pts if made if 2o in R1
ivits,48.087071,2.519794
taklons,44.262295,6.406140
firaks,32.642020,8.627015
hadsch-hallas,32.618409,5.015655
itars,28.145266,3.300642
xenos,22.391858,5.709548
nevlas,20.964750,3.936551
bescods,20.101010,4.421836
gleens,17.575188,1.041429
baltaks,15.590947,7.102002


# Faction Early Feds

In [13]:
faction_list = df_cleaned.pos_1_faction.unique()
assert(len(faction_list) == 14)

In [14]:
pd.options.display.float_format = '{:.2}'.format
for i in [3,2,1]:
    full_dict = {}
    sort_key = '% made fed in 5vp/fed round R{}'.format(i)
    for faction in faction_list:
        faction_dict = {}
        faction_df = get_faction_df(df_filtered, faction)
        fed_scoring = faction_df[faction_df['round_{}_scoring'.format(i)] == '5vp_per_fed_place']
        made_fed = faction_df[(faction_df['round_{}_scoring'.format(i)] == '5vp_per_fed_place') & (faction_df['target_score_round{}'.format(i)] >= 5)]
        faction_dict[sort_key] = len(made_fed)/len(fed_scoring) * 100
        faction_dict['if 5vp/fed scoring R{}, additional +pts if made fed'.format(i)] = made_fed.target_score.mean() - fed_scoring.target_score.mean()
        faction_dict['+pts if 5vp/fed scoring R{}'.format(i)] = fed_scoring.target_score.mean() - faction_df.target_score.mean()
        faction_dict['overall avg score'] = faction_df.target_score.mean()
        faction_dict['avg score if 5vp/fed scoring R{}'.format(i)] = fed_scoring.target_score.mean()
        faction_dict['avg score if made fed in 5vp/fed scoring R{}'.format(i)] = made_fed.target_score.mean()
        faction_dict['% players pick if 5vp/fed scoring R{}'.format(i)] = ((fed_scoring.shape[0]/faction_df.shape[0])*100)
        faction_dict['total games'] = faction_df.shape[0]
        faction_dict['games with 5vp/fed scoring R{}'.format(i)] = fed_scoring.shape[0]
        faction_dict['games fed made in 5vp/fed scoring R{}'.format(i)] = made_fed.shape[0]
        full_dict[faction] = faction_dict
    assembled_df = pd.DataFrame.from_dict(full_dict,orient='index').sort_values(by=sort_key, ascending=False)
    html_df = getTableHTML(assembled_df)
    
    with open('r{}_fed_details.html'.format(i), 'w') as fo:
        fo.write(html_df.to_html())
html_df

,% made fed in 5vp/fed round R1,"if 5vp/fed scoring R1, additional +pts if made fed",+pts if 5vp/fed scoring R1,overall avg score,avg score if 5vp/fed scoring R1,avg score if made fed in 5vp/fed scoring R1,% players pick if 5vp/fed scoring R1,total games,games with 5vp/fed scoring R1,games fed made in 5vp/fed scoring R1
ivits,74.60,4.45,-2.93,158.20,155.27,159.72,12.47,1516,189,141
gleens,20.34,-14.45,-10.46,141.28,130.82,116.38,11.09,1064,118,24
lantids,12.05,4.48,-2.03,143.45,141.42,145.90,9.22,900,83,10
xenos,4.67,-17.44,-4.26,143.51,139.24,121.80,9.08,1179,107,5
hadsch-hallas,2.73,-22.55,-5.41,145.63,140.22,117.67,9.83,1119,110,3
taklons,1.39,2.48,-0.02,160.04,160.02,162.50,10.26,1403,144,2
nevlas,0.93,33.84,1.18,148.97,150.16,184.00,10.02,1078,108,1
ambas,0.88,0.90,-3.57,151.67,148.10,149.00,9.11,1241,113,1
geodens,0.85,32.65,-3.29,143.64,140.35,173.00,9.72,1214,118,1
itars,0.63,13.71,-3.98,160.27,156.29,170.00,10.25,1542,158,1


In [15]:
df_cleaned.balance_variant.value_counts()

standard         4961
more-balanced    1610
beta              143
Name: balance_variant, dtype: int64

In [16]:
len(df_total)

NameError: name 'df_total' is not defined

In [ ]:
[_ for _ in list(df_total.columns) if 'pos_1' in _]

In [ ]:
df = df_total.loc[df_total.average_elo > 400]

In [ ]:
df1 = df.loc[(df.pos_1_faction=='terrans') & (df.round_6_scoring=='4vp_per_gaia_place'), 'pos_1_score']
df2 = df.loc[(df.pos_2_faction=='terrans') & (df.round_6_scoring=='4vp_per_gaia_place'), 'pos_2_score']
df3 = df.loc[(df.pos_3_faction=='terrans') & (df.round_6_scoring=='4vp_per_gaia_place'), 'pos_3_score']
df4 = df.loc[(df.pos_4_faction=='terrans') & (df.round_6_scoring=='4vp_per_gaia_place'), 'pos_4_score']
# print(df1.mean(), df2.mean(), df3.mean(), df4.mean())
# print(df1.size, df2.size, df3.size, df4.size)
avg_4vp = (df1.sum() + df2.sum() + df3.sum() + df4.sum())/(df1.size + df2.size + df3.size + df4.size)
df1 = df.loc[(df.pos_1_faction=='terrans'), 'pos_1_score']
df2 = df.loc[(df.pos_2_faction=='terrans'), 'pos_2_score']
df3 = df.loc[(df.pos_3_faction=='terrans'), 'pos_3_score']
df4 = df.loc[(df.pos_4_faction=='terrans'), 'pos_4_score']
# print(df1.mean(), df2.mean(), df3.mean(), df4.mean())
# print(df1.size, df2.size, df3.size, df4.size)
total_games = df1.size + df2.size + df3.size + df4.size
avg_all = (df1.sum() + df2.sum() + df3.sum() + df4.sum())/total_games
print(avg_4vp, avg_all, total_games)

In [ ]:
df1 = df.loc[(df.pos_1_faction=='terrans') & (df.round_5_scoring=='4vp_per_gaia_place'), 'pos_1_score']
df2 = df.loc[(df.pos_2_faction=='terrans') & (df.round_5_scoring=='4vp_per_gaia_place'), 'pos_2_score']
df3 = df.loc[(df.pos_3_faction=='terrans') & (df.round_5_scoring=='4vp_per_gaia_place'), 'pos_3_score']
df4 = df.loc[(df.pos_4_faction=='terrans') & (df.round_5_scoring=='4vp_per_gaia_place'), 'pos_4_score']
# print(df1.mean(), df2.mean(), df3.mean(), df4.mean())
# print(df1.size, df2.size, df3.size, df4.size)
avg_4vp = (df1.sum() + df2.sum() + df3.sum() + df4.sum())/(df1.size + df2.size + df3.size + df4.size)
df1 = df.loc[(df.pos_1_faction=='terrans'), 'pos_1_score']
df2 = df.loc[(df.pos_2_faction=='terrans'), 'pos_2_score']
df3 = df.loc[(df.pos_3_faction=='terrans'), 'pos_3_score']
df4 = df.loc[(df.pos_4_faction=='terrans'), 'pos_4_score']
# print(df1.mean(), df2.mean(), df3.mean(), df4.mean())
# print(df1.size, df2.size, df3.size, df4.size)
total_games = df1.size + df2.size + df3.size + df4.size
avg_all = (df1.sum() + df2.sum() + df3.sum() + df4.sum())/total_games
print(avg_4vp, avg_all, total_games)

In [ ]:
list(range(1,4))

In [ ]:
display(get_faction_dddf(df_cleaned,'ivits')['target_score'].mean())
display(get_faction_df(df_cleaned,'lantids')['target_score'].mean())
display(get_faction_df(df_cleaned,'ivits')['target_bid'].mean())
display(get_faction_df(df_cleaned,'lantids')['target_bid'].mean())